In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd
import warnings
import numpy as np
from fuzzywuzzy import process, fuzz
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
def fuzzy_nama(df, param_1=fuzz.ratio, param_2=fuzz.token_set_ratio):
    from fuzzywuzzy import process, fuzz

    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'urut'})
    
    df['len'] = df['Nama'].str.len()
    df['word'] = df['Nama'].str.split().str.len()
    df['dummy'] = df['Nama'].copy()
    
    def remove_char(x):
        remove = []
        for i in x.split():
            if len(i) > 1:
                remove.append(i)
        remove = [' '.join(remove)]
        return remove[0]
    
    df['dummy'] = df['dummy'].apply(lambda x : remove_char(x))
    df = df.sort_values(by=['len'], ascending=False)
    df_Nama = df.dummy.tolist()
    Alias = df_Nama  

    list_1 = []
    list_Alias = list(dict.fromkeys(Alias))
    for x in Alias:
        highest = process.extract(x,list_Alias,scorer=param_1, limit=2)
        if len(x.split()) == 1:
            continue            
        elif int(highest[1][1]) > 80:
            list_1.append(highest[1][0])
        else :
            list_1.append(highest[0][0])

    list_1 = list( dict.fromkeys(list_1))
    
    list_2 = []
    for x in Alias:
        highest = process.extractOne(x,list_1, score_cutoff = 100)
        if highest != None:
            list_2.append(highest[0])
    #
    list_2 = list(dict.fromkeys(list_2))    
    x = []
    for i in list_2:
        a = i.split()
        x.append(a)
    for i in range(len(x)):
        if i == 0:
            y = x[0]
        else :
            y = y + x[i]
    list_3 = []
    for x in y:
        highest = process.extract(x,y,scorer=param_1, limit=2)
        if int(highest[1][1]) > 70:
            list_3.append(highest[1][0])
        else :
            list_3.append(highest[0][0])    
    
    dicti = pd.DataFrame()
    dicti['y'] = y
    dicti['x'] = list_3
    dicti['cek'] = np.where(dicti["x"] == dicti["y"], True, False)
    dicti = dicti.drop_duplicates(subset = ["y"])
    dicti['len'] = dicti['y'].str.len()
    dicti = dicti[(dicti['cek'] == False)]
    dicti = dicti.sort_values(by=['len'], ascending=False)
    dicti = dicti.reset_index(drop=True)
    
    final = []
    for i in list_2:
        list_k = []
        for k in i.split():
            for j in range(len(dicti)):
                if k == dicti['y'][j]:
                    final_ = str(i).replace(str(dicti['y'][j]), str(dicti['x'][j]))
                    list_k = list(dict.fromkeys(list_k))
                    list_k.append(final_)
            if k not in dicti['y'].tolist() :
                list_k.append(i)
            list_k = list(dict.fromkeys(list_k))
        final.append(list_k)

    list_3 = []
    for i in final:
        list_3.append(i[0])

    list_3 = list(dict.fromkeys(list_3))
    list_4 = []    
    for x in Alias:
        highest = process.extract(x,list_3,scorer=param_2, limit=2)
        list_4.append(highest[0][0])     

    list_4 = list( dict.fromkeys(list_4))
    list_5 = []
    similirity = []
    for x in df_Nama:
        highest = process.extractOne(x,list_4)
        if len(x.split()) == 1:
            list_5.append(x)
            similirity.append(100)
        else :
            list_5.append(highest[0])
            similirity.append(highest[1])

    df['Alias'] = list_5
    df['similirity'] = similirity
    df = df.sort_values(by=['urut'], ascending=True)
    df = df.drop(["urut", 'len','dummy','word'], axis =1 )
    
    return df

In [ ]:
Nama = {'Nama': 
        ['M Haykal Narendra','Muhammad Haykal Narendra','M Haykal N','Haykal Narendra', 'Haykal N',
        'Lintang Gilang Pratama', 'Lintang Gilang', 'Lintang G P', 'Lintang', 'Lintang Gilang P',
        'Rindhy','Rindhyy Utami Muris', 'Rindhy Utami M', 'Rindi Utami ', 'Rindhy Utami',
        'Dimas Galih Satrio', 'Dimass Galih', 'Dimas G S', 'Dimas Galih S', 'Dimas',
        'Agung', 'Agung Kurniawan', 'Agung Kurniawan Sastro', 'Agung K S', 'Agungg Kurniawan',
        'Haykal Lintang Rindhy Dimas Agung','Jokowi','Fiqaha', 'Fiqaha Hawalin'],
        }
df = pd.DataFrame(Nama, columns = ['Nama'])
df

,Nama
0,M Haykal Narendra
1,Muhammad Haykal Narendra
2,M Haykal N
3,Haykal Narendra
4,Haykal N
5,Lintang Gilang Pratama
6,Lintang Gilang
7,Lintang G P
8,Lintang
9,Lintang Gilang P


In [ ]:
df = fuzzy_nama(df, param_1=fuzz.ratio, param_2=fuzz.token_set_ratio) 
#token_sort_ratio #token_set_ratio 
#partial_token_sort_ratio 
#partial_ratio

df

,Nama,Alias,similirity
0,M Haykal Narendra,Muhammad Haykal Narendra,90
1,Muhammad Haykal Narendra,Muhammad Haykal Narendra,100
2,M Haykal N,Haykal,100
3,Haykal Narendra,Muhammad Haykal Narendra,90
4,Haykal N,Haykal,100
5,Lintang Gilang Pratama,Lintang Gilang Pratama,100
6,Lintang Gilang,Lintang Gilang Pratama,90
7,Lintang G P,Lintang,100
8,Lintang,Lintang,100
9,Lintang Gilang P,Lintang Gilang Pratama,90
